In [44]:
# EmotiQuest Chatbot - Simplified Prototype Version
!pip install gradio diffusers transformers accelerate safetensors --quiet

import gradio as gr
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from diffusers import StableDiffusionPipeline
import torch
import random
from PIL import Image
import re
import uuid
import os

In [45]:
# Load GPT-2 model
model_id = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)
text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Load Stable Diffusion for images
pipe = StableDiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2-1")
pipe = pipe.to("cuda" if torch.cuda.is_available() else "cpu")

# Predefined fields to replace unreliable GPT generations
fantasy_races = ["Elf", "Dwarf", "Dragonborn", "Tiefling", "Aasimar", "Orc", "Halfling"]
fantasy_classes = ["Warrior", "Mage", "Rogue", "Druid", "Paladin", "Ranger", "Bard"]
personalities = [
    "Brave, Loyal, Curious",
    "Mysterious, Calm, Calculating",
    "Chaotic, Energetic, Bold",
    "Kind, Gentle, Wise",
    "Witty, Clever, Observant"
]
special_traits = [
    "Can summon spirit animal",
    "Heals allies in moonlight",
    "Immune to mind control",
    "Inspires courage with a song",
    "Can sense danger from afar"
]
favorite_weapons = ["Longsword", "Magic Staff", "Daggers", "Bow and Arrows", "Battle Axe"]
armors = ["Leather Armor", "Chainmail", "Robes of Mysticism", "Dragonhide Armor", "Cloak of Shadows"]
mounts = ["Griffin", "Shadow Horse", "Dire Wolf", "Phoenix", "Magical Elk"]
alignments = ["Lawful Good", "Neutral", "Chaotic Good", "Chaotic Neutral", "True Neutral"]

# Generate a fantasy name based on mood
def generate_name(mood):
    mood = mood.lower()
    prefixes = {
        "happy": ["Sun", "Joy", "Fel", "Lio"],
        "angry": ["Grim", "Vex", "Doom", "Thar"],
        "mysterious": ["Shae", "Vel", "Nyx", "Zan"],
    }
    suffixes = ["wyn", "dor", "mir", "thorn"]
    pre = random.choice(prefixes.get(mood.split()[0], ["Ara", "Fin", "Lor"]))
    suf = random.choice(suffixes)
    return pre + suf

# Full character sheet (without backstory)
def generate_character_bio(mood):
    name = generate_name(mood)
    race = random.choice(fantasy_races)
    char_class = random.choice(fantasy_classes)
    personality = random.choice(personalities)
    trait = random.choice(special_traits)
    weapon = random.choice(favorite_weapons)
    armor = random.choice(armors)
    mount = random.choice(mounts)
    alignment = random.choice(alignments)

    # Generate stats
    stats = {
        "Strength": random.randint(8, 18),
        "Dexterity": random.randint(8, 18),
        "Constitution": random.randint(8, 18),
        "Intelligence": random.randint(8, 18),
        "Wisdom": random.randint(8, 18),
        "Charisma": random.randint(8, 18)
    }
    stats_str = "\n".join([f"- {k}: {v}" for k, v in stats.items()])

    return f"""Name: {name}
Race: {race}
Class: {char_class}
Personality: {personality}
Alignment: {alignment}
Favorite Weapon: {weapon}
Armor: {armor}
Mount: {mount}
Special Trait: {trait}
Stats:\n{stats_str}"""

# Generate portrait only
def generate_character_image_only(mood, name):
    prompt = f"Fantasy portrait of {name}, {mood} mood, highly detailed, concept art"
    image = pipe(prompt).images[0]
    return image

# Chatbot logic
def chatbot_response(message, history):
    if not message.strip():
        return "Please describe your mood to generate a character!"
    try:
        bio = generate_character_bio(message)
        name_match = re.search(r"Name:\s*(.*)", bio)
        name = name_match.group(1).strip() if name_match else generate_name(message)
        img = generate_character_image_only(message, name)
        img_path = f"/tmp/{uuid.uuid4()}.png"
        img.save(img_path)
        return {"text": bio, "files": [img_path]}
    except Exception as e:
        return f"Error: {str(e)}"

# Gradio App UI
with gr.Blocks() as app:
    gr.Markdown("## 🌟 Chat with EmotiQuest! Start with a Mood ➜ Get a Character!")
    gr.ChatInterface(
        chatbot_response,
        examples=["happy", "mysterious", "angry", "playful rogue"],
        title="🎮 Let's EmotiQuest! What's Your Mood Today?"
    )

app.launch()


Device set to use cuda:0


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/gradio/chat_interface.py:334: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7886
Kaggle notebooks require sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://e4c1d5182a71d8be46.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


  0%|          | 0/50 [00:00<?, ?it/s]